In [2]:
# pytorch library
import torch

import torch.nn as nn
import torch.optim as optim
# transforms perform a set of operations to an individual item (image) so that it is usable by the model
import torchvision.transforms as transforms
# Tools for creating datasets from folders
from torchvision import datasets
# default_loader enables basic loading of images to be used in the CustomDataset class
from torchvision.datasets.folder import default_loader
# DataLoader creates batches of data in a format the model can use
# Dataset is a general class that allows the retrieval of individual pieces of data via the DataLoader
# All classes that inherit from Dataset must implement the __getitem__ and __len__ 
from torch.utils.data import DataLoader, Dataset
import pandas as pd


### Setup transform and define CustomDataset class for iterable datasets

In [3]:
transform=transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Use the train.csv and val.csv csv files to create two datasets
class CustomDataset(Dataset):
    # csv_file is a file path, root_dir is the root of the dataset, transform is an optional instantiation of transforms.Compose
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    # How many items are in the dataset
    def __len__(self):
        return len(self.data)
    
    # Get a specific item from the dataset by index
    def __getitem__(self, idx):
        img_name = self.data.iloc[idx,0] # column 0 contains file paths
        img_path = f"{self.root_dir}/{img_name}"
        # A basic image loader that retruns a Python Imaging Library (PIL) object
        image = default_loader(img_path)
        label = self.data.iloc[idx,1] # column 1 contains image class

        # If there is a transform function, transform the image
        if self.transform:
            image = self.transform(image)

        return image, label

### Setup data

In [4]:
data_path = "../datasets/bean-leaf-lesions/"

# CSV file paths
train_csv = data_path + 'train.csv'
val_csv = data_path + 'val.csv'

# Create custom datasets
train_dataset = CustomDataset(csv_file=train_csv, root_dir=data_path, transform=transform)
val_dataset = CustomDataset(csv_file=val_csv, root_dir=data_path, transform=transform)

# Create Dataloaders
batch_size=32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) 

### Create NN

In [ ]:
# nn.Module is the base class for all NN modules in pytorch
class CNN(nn.Module):
    def __init__(self):
        # CNN class inherits from nn.Module
        # Inheriting allows the use of functions from the parent class
        super(CNN, self).__init__()
        # Input = RGB, Output = number of filters/kernels in the layer, 
        # kernel_size = height&width of convolutional window, padding = number of 0 pixels added to edge of image
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 16, kernel_size= 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)